Using Newton's second law, we set up equations of motion for each pendulum and isolate acceleration terms:

$$ m_1 \ddot{x_1} = \frac{-m_1 g}{l_1}x_1 + k(x_2 - x_1) \leftrightarrow \ddot{x_1} = \frac{-g}{l_1}x_1 + \frac{k}{m_1}(x_2 - x_1)$$
$$ m_2 \ddot{x_2} = \frac{-m_2 g}{l_2}x_1 - k(x_2 - x_1) \leftrightarrow \ddot{x_2} = \frac{-g}{l_2}x_2 - \frac{k}{m_1}(x_2 - x_1)$$

We represent the system in matrix form:
$$
\begin{bmatrix}
\ddot{x}_1 \\
\ddot{x}_2
\end{bmatrix}
= - 
\begin{bmatrix}
\frac{k}{m_1} + \frac{g}{l_1} & -\frac{k}{m_1} \\
-\frac{k}{m_2} & \frac{k}{m_2} + \frac{g}{l_2}
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
$$

The eigenvalues correspond to the angular frequencies of the system's normal modes. We determine the normal mode frequencies by creating and solving the characteristic equation:

$$
\text{det}
\left(
\begin{bmatrix}
\frac{k}{m_1} + \frac{g}{l_1} - \omega^2 & -\frac{k}{m_1} \\
-\frac{k}{m_2} & \frac{k}{m_2} + \frac{g}{l_2} - \omega^2
\end{bmatrix}
\right) = 0
$$

In [50]:
m₁ = 2.5
m₂ = 5.0
l₁ = 0.90
l₂ = 0.45
g = 9.81
k = 10
x₀ = [0.09; 0.045]
v₀ = [0; 0]
t = 5.3


using LinearAlgebra

NearZero(x) = abs(x) < 1e-12 ? 0.0 : x
PhaseShift(Re, Im) =  Re < 1 ? atan(Im/Re) + π : atan(Im/Re)

function solve_EoM(m₁, m₂, l₁, l₂, g, k, x₀, v₀, t)

     A = [(k/m₁ + g/l₁) -k/m₁;
          -k/m₂ (k/m₂ + g/l₂)]

     e₁ = eigen(A).vectors[:, 1]
     e₂ = eigen(A).vectors[:, 2]
     e₁ = e₁[1] > 0 ? e₁ : -e₁
     e₂ = e₂[1] > 0 ? e₂ : -e₂
     ω₁ = √eigen(A).values[1]
     ω₂ = √eigen(A).values[2]
     
     ReC₁ = e₁ ⋅ x₀
     ReC₂ = e₂ ⋅ x₀
     ImC₁ = (-1/ω₁) * (e₁ ⋅ v₀)
     ImC₂ = (-1/ω₂) * (e₂ ⋅ v₀)

#     for value in [ReC₁,ReC₂,ImC₁,ImC₂, e₁ , e₂]
#          value = NearZero(value)
#     end

     A₁ = √(ReC₁^2 + ImC₁^2)
     A₂ = √(ReC₂^2 + ImC₂^2)
     ϕ₁ = NearZero(PhaseShift(ReC₁,ImC₁))
     ϕ₂ = NearZero(PhaseShift(ReC₂,ImC₂))

     println("Calculated Values:")
     varNames = ["ω₁", "ω₂", "ReC₁", "ReC₂", "ImC₁", "ImC₂", "A₁", "A₂", "ϕ₁", "ϕ₂"]
     varValues = [ω₁, ω₂, ReC₁, ReC₂, ImC₁, ImC₂, A₁, A₂, ϕ₁, ϕ₂]
     for i in 1:10
          println(varNames[i], ": ", round(varValues[i],digits=2))
     end

     x₁(t) = A₁ * cos(ω₁ * t + ϕ₁) * e₁[1] + A₂ * cos(ω₂ * t + ϕ₂) * e₂[1]
     x₂(t) = A₁ * cos(ω₁ * t + ϕ₁) * e₁[2] + A₂ * cos(ω₂ * t + ϕ₂) * e₂[2]

     InitialPositionVector = [round(x₁(0), digits=3); round(x₂(0), digits=3)]
     LaterPositionVector = [round(x₁(t), digits=3); round(x₂(t), digits=3)]
     initial_positions_agree = all(isapprox.(InitialPositionVector, x₀, atol=1e-12))

     println("\nAt t = 0s, calculated x(t) is $InitialPositionVector" * 
     (initial_positions_agree ? ", which agrees with given x₀" : ", which does NOT agree with given x₀") * ", of $x₀")
     println("At t = " * string(t) * "s, calculated x(t) is " * string(LaterPositionVector))
end

solve_EoM(m₁, m₂, l₁, l₂, g, k, x₀, v₀, t)

Calculated Values:
ω₁: 3.75
ω₂: 4.96
ReC₁: 0.1
ReC₂: -0.01
ImC₁: -0.0
ImC₂: -0.0
A₁: 0.1
A₂: 0.01
ϕ₁: 3.14
ϕ₂: 3.14

At t = 0s, calculated x(t) is [-0.098, -0.013], which does NOT agree with given x₀, of [0.09, 0.045]
At t = 5.3s, calculated x(t) is [-0.05, -0.007]


In [65]:
using Pkg
Pkg.add("DifferentialEquations")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [70]:
using LinearAlgebra

function solve_coupled_pendulums(m1, l1, m2, l2, k, g, x01, x02, v01, v02)
    # Create the modified stiffness matrix K'
    K_prime = [k/m1 + g/l1 -k/m1; -k/m2 k/m2 + g/l2]

    # Solve the eigenvalue problem
    eigenvalues, V = eigen(K_prime)

    # The eigenvalues correspond to ω^2, take the square root to get the frequencies
    ω = sqrt.(eigenvalues)

    # Normalize the eigenvectors
    for i in 1:size(V, 2)
        V[:, i] /= norm(V[:, i])
    end

    # Solve for the coefficients A and B in the general solution using the initial conditions
    A = V \ [x01; x02]
    B = V \ [v01 / ω[1]; v02 / ω[2]]

    # Define the functions x1(t) and x2(t) for the displacements
    function x1(t)
        A[1] * cos(ω[1] * t) + B[1] * sin(ω[1] * t)
    end

    function x2(t)
        A[2] * cos(ω[2] * t) + B[2] * sin(ω[2] * t)
    end

    return x1, x2
end

# Test the function with given parameters and initial conditions
m1 = 1.0
l1 = 1.0
m2 = 1.5
l2 = 1.2
k = 100.0
g = 9.81
x01 = 0.1
x02 = 0.2
v01 = 0.0
v02 = 0.0

x1, x2 = solve_coupled_pendulums(m1, l1, m2, l2, k, g, x01, x02, v01, v02)

# Example evaluation at t = 1.0 second
t = 0
println("Position of pendulum 1 at time t = $t: ", x1(t))
println("Position of pendulum 2 at time t = $t: ", x2(t))

Position of pendulum 1 at time t = 0: 0.22638212208219707
Position of pendulum 2 at time t = 0: -0.07104465187358079
